# Описание проекта

На основе данных оператора мобильной связи "Мегалайн" о поведении клиентов, которые уже перешли на новые тарифы, построить систему способную проанализировать клиентов, использующих старые тарифы и предложить на пододящий новый

# Доп. сведения о проекте/данных
- предобработку делать не нужно, данные уже приведены к корректному виду
- построить модель с метрикой accurace > 0.75
- каждый объект - информация о поведении одного пользователя за месяц

# Шаг 1. Открытие данных и анализ общей информации о данных

In [1]:
# импортируем необходимые библиотеки и задаем настройки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
# открываем файл с данными
try:
    data = pd.read_csv('/datasets/users_behavior.csv')
except FileNotFoundError:
    data = pd.read_csv('users_behavior.csv')

In [3]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Описание данных

- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Исходя из описания данных стоит изменить тип данных в calls и messages на целочисленный

In [5]:
data['calls'] = data['calls'].astype('int16')
data['messages'] = data['messages'].astype('int16')
data['is_ultra'] = data['is_ultra'].astype('int16')

Посмотрим на общее распределение данных

In [6]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


<b>Вывод: </b>данные распределены нормально, пропусков нет. Небольшой перекос в данных есть в столбце messages

# Шаг 2. Разделение выборки на обучающую, валидационную, тестовую

Разделить исходные данные на обучающую, валидационную и тестовую выборки

In [7]:
features = data.drop('is_ultra', axis=1)
target = data['is_ultra']

In [8]:
features_temp, features_test, target_temp, target_test = train_test_split(
    features, 
    target, 
    test_size=0.2, 
    random_state=12345)

In [9]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features_temp, 
    target_temp, 
    test_size=0.2, 
    random_state=12345)

# Шаг 3. Исследования качества разных моделей

Будем исследовать качество следующих моделей:
- Решающее дерево
- Случайный лес
- Логистическая регрессия

## Решающее дерево

In [10]:
%%time
best_result = 0
best_depth = 0
best_criterion = ''
best_model_decision_tree = None
best_min_samples_split = 0
best_min_samples_leaf = 0
for criterion in ['gini', 'entropy']:
    for min_samples_split in range(2, 6):
        for min_samples_leaf in range(1, 10):
            for depth in range(1, 10):
                model = DecisionTreeClassifier(
                    random_state=12345, 
                    max_depth=depth, 
                    criterion=criterion,
                    min_samples_split = min_samples_split,
                    min_samples_leaf = min_samples_leaf
                )
                model.fit(
                    features_train, 
                    target_train
                )
                result = model.score(
                    features_valid, 
                    target_valid
                )
                if result > best_result:
                    best_result = round(result, 3)
                    best_depth = depth
                    best_criterion = criterion
                    best_model_decision_tree = model
                    best_min_samples_leaf = min_samples_leaf
                    best_min_samples_split = min_samples_split
print('Accuracy:\t\t', best_result, 
      '\nmax_depth:\t\t', best_depth, 
      '\ncriterion:\t\t', best_criterion,
      '\nmin_samples_leaf\t', best_min_samples_leaf,
      '\nbest_min_samples_split\t', best_min_samples_split)
print()

Accuracy:		 0.781 
max_depth:		 8 
criterion:		 entropy 
min_samples_leaf	 3 
best_min_samples_split	 2

CPU times: user 2.6 s, sys: 0 ns, total: 2.6 s
Wall time: 2.6 s


## Случайный лес

In [11]:
%%time
best_result = 0
best_est = 0
best_criterion = ''
best_depth = 0
best_model_random_forest = None
best_min_samples_leaf = 0
best_min_samples_split = 0
for est in range(1, 11):
    for criterion in ['gini', 'entropy']:
        for min_samples_leaf in range(2, 10):
            for min_samples_split in range(2, 10):
                for depth in range(1, 10):
                    model = RandomForestClassifier(
                        random_state=12345, 
                        n_estimators=est, 
                        criterion=criterion, 
                        max_depth=depth,
                        min_samples_leaf=min_samples_leaf,
                        min_samples_split=min_samples_split
                    )
                    model.fit(
                        features_train, 
                        target_train
                    )
                    result = model.score(
                        features_valid, 
                        target_valid
                    )
                    if result > best_result:
                        best_result = round(result, 3)
                        best_est = est
                        best_criterion = criterion
                        best_depth = depth
                        best_model_random_forest = model
                        best_min_samples_leaf = min_samples_leaf
                        best_min_samples_split = min_samples_split
print('Accuracy:\t\t', best_result, 
      '\nmax_depth:\t\t', best_depth, 
      '\ncriterion:\t\t', best_criterion,
      '\nmin_samples_leaf\t', best_min_samples_leaf,
      '\nbest_min_samples_split\t', best_min_samples_split, 
      '\nestimators:\t\t', best_est)
print()

Accuracy:		 0.796 
max_depth:		 9 
criterion:		 gini 
min_samples_leaf	 2 
best_min_samples_split	 4 
estimators:		 9

CPU times: user 2min 1s, sys: 227 ms, total: 2min 1s
Wall time: 2min 1s


## Логистическая регрессия

In [12]:
%%time
best_model_logistic_reg = LogisticRegression(random_state=12345)
best_model_logistic_reg.fit(features_train, target_train)
result = best_model_logistic_reg.score(features_valid, target_valid)
print('Accuracy: ', result)

Accuracy:  0.7165048543689321
CPU times: user 18.9 ms, sys: 18 µs, total: 18.9 ms
Wall time: 18.5 ms


<b>Промежуточные выводы о качестве моделей на валидационной выборке:</b>

Лучший результат (в прочем это было очевидно) показала модель случайного леса (accuracy = 0.796)

Результат решающего дерева accuracy = 0.781. Не намного меньше.

Но время обучения для модели случайного леса составило примерно 2 минуты, а для модели решающего дерева всего лишь около 2.5 секунд. С данной точки зрения решающее дерево безоговорочный лидер.

Модель логистической регресии даже не смогла удовлетворить условиям задачи.

# Шаг 4. Проверка лучшей модели на тестовой выборке

Лучшая модель - случайный лес

In [13]:
result = best_model_random_forest.score(features_test, target_test)
print('Accuracy лучшей модели случайного леса на тестовой выборке: ', round(result, 3))

Accuracy лучшей модели случайного леса на тестовой выборке:  0.796


In [14]:
result = best_model_decision_tree.score(features_test, target_test)
print('Accuracy лучшей модели решающего дерева на тестовой выборке: ', round(result, 3))

Accuracy лучшей модели решающего дерева на тестовой выборке:  0.771


<b>Выводы по итогам работы с тестовой выборкой:</b>

Модель случайного леса на валидационной выборке показала accuracy = 0.796. На тестовой выборке 0.796

Модель решающего дерева на валидационной выборке показала accuracy = 0.781. На тестовой выборке 0.771

# Шаг 5. Проверка модели на "вменяемость"

Проверка модели на адекватность

In [15]:
dummy_model = DummyClassifier(strategy='most_frequent', random_state=12345)
dummy_model.fit(features_train, target_train)
result_dummy = dummy_model.score(features_valid, target_valid)
print('Accuracy Dummy модели: ', round(result_dummy, 3))

Accuracy Dummy модели:  0.676


# Общий вывод:

1. Данные успешно загружены и проанализированы.
2. Данные разделены на тренировочную, валидационную и тестовые выборки в соотношении 3:1:1
3. Для анализа качества моделей выбраны следующие: решающее дерево, случайный лес и логистическая регрессия
4. Лучшая модель 
- по качеству - случайный лес, но затрачивает огромное время на обучение (подбору гиперпараметров)
- по времени - решающее дерево, accuracy чуть ниже, но время обучения в разы меньше
- логистическая регрессия не смогла показать допустимый задачей результат
5. При тестировании моделей на тестовой выборке
- случайный лес показал аналогичное значение accuracy
- решающее дерево показало результат чуть хуже чем на тренировочной выборке
6. Проведен тест на вменяемость модели с помощью DummyClassifier. Accuracy = 0.676. В наших моделях результат оказался повыше.